In [2]:
from obspy import read, read_events, read_inventory
import os
import sys
from glob import glob
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import coincidence_trigger
from obspy.core.event.catalog import Catalog
from obspy.core.event.event import Event
from obspy.core.event.magnitude import StationMagnitude, Magnitude, StationMagnitudeContribution, Amplitude
from obspy.core.event.base import QuantityError, TimeWindow, WaveformStreamID, Comment
from helper_functions import *

In [6]:
cat = read_events("detections/20200310/bhdetections_20200310_*.xml")
catm = Catalog()
for ev in cat:
    if len(ev.magnitudes):
        catm += ev
        
print(catm)

Event:	 | -2.2129028140039715 Mw_spectral

	        resource_id: ResourceIdentifier(id="smi:local/b14b88d0-63e3-44f5-a10e-3224168954b5")
	               ---------
	              picks: 22 Elements
	         amplitudes: 22 Elements
	         magnitudes: 2 Elements
	 station_magnitudes: 2 Elements
Event:	 | -3.2416876991087893 Mw_time

	        resource_id: ResourceIdentifier(id="smi:local/d2394161-3d0e-4b3a-b312-f70c7a9f0aa1")
	               ---------
	              picks: 15 Elements
	         amplitudes: 14 Elements
	         magnitudes: 1 Elements
	 station_magnitudes: 1 Elements
Event:	 | -3.1448022867233516 Mw_spectral

	        resource_id: ResourceIdentifier(id="smi:local/768366d7-0c3d-41c6-91fc-320d8b2c62be")
	               ---------
	              picks: 14 Elements
	         amplitudes: 14 Elements
	         magnitudes: 2 Elements
	 station_magnitudes: 2 Elements
Event:	 | -2.374314636492673 Mw_spectral

	        resource_id: ResourceIdentifier(id="smi:local/15c6451b-dd46-4d

In [110]:
def get_stream(ev):

    starttime = min([p.time for p in ev.picks]) - 1.0
    endtime = min([p.time for p in ev.picks]) + 2.0
    stations = list(set([p.waveform_id.station_code for p in ev.picks]))
    DATA_DIR_ROOT = "/home/gilbert_lab/cami_frs/borehole_data/sac_daily_nez_500Hz/"
    datestr = "20200310"

    stream = read(os.path.join(os.path.join(DATA_DIR_ROOT, datestr), "*.sac"), starttime=starttime, endtime=endtime)
    stream.detrend("demean")
    stream.detrend("linear")
    
    return stream



In [36]:
nmagn = [len(ev.station_magnitudes) for ev in catm]
catm_sorted = [ev for _, ev in sorted(zip(nmagn, catm), key=lambda x: x[0], reverse=True)]

ev = catm_sorted[0]
print(ev) 

for amp in ev.amplitudes:
    #print(amp)
    if amp.type == "A":
        #print(amp)
        print("\tSNR = %.2f" % amp.snr)
        print("\t%s" % amp.waveform_id.station_code)
        #print("\t%s" % amp.time_window)
        
for magn in ev.magnitudes:
    
    print("Magnitude: \n\t Type: %s\n\t Value= %.2f +/ %.2f" % (magn.magnitude_type, magn.mag, magn.mag_errors.uncertainty))

    for contrib in magn.station_magnitude_contributions:
        sid = contrib.station_magnitude_id
        smag = [m for m in ev.station_magnitudes if m.resource_id == sid][0]
        print(smag.mag)
        print(smag.comments[-1].text)
        #for com in smag.comments:
        #    print("\t%s" % com.text)


Event:	 | -2.374314636492673 Mw_spectral

	        resource_id: ResourceIdentifier(id="smi:local/15c6451b-dd46-4d2a-ae5f-8dc60e7a6988")
	               ---------
	              picks: 20 Elements
	         amplitudes: 20 Elements
	         magnitudes: 2 Elements
	 station_magnitudes: 10 Elements
Magnitude
	                     resource_id: ResourceIdentifier(id="smi:local/bf028e21-cd8f-4171-9647-364f91fe2f21")
	                             mag: -2.374314636492673 [uncertainty=0.285366669417]
	                  magnitude_type: 'Mw_spectral'
	                   station_count: 5
	                            ---------
	 station_magnitude_contributions: 5 Elements


NameError: name 'mag' is not defined